In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import time
import dotenv
import os

dotenv.load_dotenv()

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

In [ ]:
dir = "./files/"
file_list = os.listdir(dir)

In [ ]:
print(file_list)

### Part1-1. 문서 기반 챗봇

In [ ]:
# 텍스트 스플리터 생성

splitter = RecursiveCharacterTextSplitter(chunk_size=300, 
                                               chunk_overlap=50, 
                                               separators=["\n\n"])

In [ ]:
# PDF 로더 생성

loader = PyPDFLoader(dir+file_list[0])

In [ ]:
# PDF 문서 로드 및 스플릿

split_docs = loader.load_and_split(text_splitter=splitter)

In [ ]:
# 벡터스토어 생성

vector_store = FAISS.from_documents(embedding=embeddings, documents=split_docs)

In [ ]:
# 리트리버 생성

retriever = vector_store.as_retriever()

In [ ]:
# 프롬프트 템플릿

prompt = ChatPromptTemplate([
    ("system", "문서 : {context}\n\n"
               "당신은 문서에서 원하는 정보를 찾는 10년차 데이터 전문가입니다. 항상 문서를 기반하여 응답해주세요\n"
               "문서에서 응답을 찾을 수 없는 경우 '문서에서 응답을 찾을 수 없습니다.' 라고 답변하세요."), 
    ("user", "{query}")

])

In [ ]:
# 문서 포맷팅

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# 체인 생성

chain = {"context": retriever | RunnableLambda(format_docs),
         "query": RunnablePassthrough()} | prompt | llm

In [ ]:
query = "2025년엔 어떤 부분을 준비해야할까?"

In [ ]:
result = chain.invoke(query)

In [ ]:
print(result.content)

In [ ]:
result = chain.stream(query)

In [ ]:
for step in result:
    print(step.content, sep="", end="")
    time.sleep(0.03)

In [ ]:
result = chain.invoke("오늘 서울 날씨는 어때?")

In [ ]:
print(result.content)

In [ ]:
result = chain.invoke("미국의 인구수는 몇 명정도 돼?")

In [ ]:
print(result.content)

### Part1-2 고객 응대 챗봇

In [ ]:
# PDF 로더 생성

loader = PyPDFLoader(dir+file_list[1])

In [ ]:
# PDF 문서 로드 및 스플릿

split_docs = loader.load_and_split(text_splitter=splitter)

In [ ]:
# 벡터스토어 생성

vector_store = # 벡터스토어를 생성해주세요.

In [ ]:
# 리트리버 생성

retriever = # 리트리버를 생성해주세요.

In [ ]:
# 프롬프트 템플릿

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
    context : {context}

    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    ---
    ### 규칙 ###
    1. 주어진 context만을 이용하여 답변해야합니다. 
    2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    4. 항상 친절한 말투로 응대합니다.
    5. 하이퍼 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요. 아래 예시를 참고하여 서식을 맞추세요.
    
    **하이퍼 링크 예시**
    5-1. [스타벅스 구역삼사거리점](https://naver.me/FV7K6xTM) 입니다.
    5-2. [화목순대국](https://naver.me/FQVGK6TZ) 입니다.
    5-3. [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5) 입니다.
    ---
    """),
    ("human", "{query}")
])

In [ ]:
# 체인 생성

chain = # [Your Chain]

In [ ]:
query = "개강하는 날짜 알려주세요."

In [ ]:
result = chain.stream(query)

In [ ]:
for step in result:
    print(step.content, sep="", end="")
    time.sleep(0.03)

In [ ]:
query = "오프라인으로도 수강할 수 있나요?"

In [ ]:
result = chain.stream(query)

In [ ]:
for step in result:
    print(step.content, sep="", end="")
    time.sleep(0.03)

In [ ]:
query = "졸업하면 어떤걸 할 수 있나요?"

In [ ]:
result = chain.stream(query)

In [ ]:
for step in result:
    print(step.content, sep="", end="")
    time.sleep(0.03)

In [ ]:
# 답변하면 안되는 질문

query = "서울에서 부산까지는 얼마나 걸리나요?"

In [ ]:
result = chain.stream(query)

In [ ]:
for step in result:
    print(step.content, sep="", end="")
    time.sleep(0.03)